In [ ]:
############# 小甜豆 ##############
import requests

headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36",
    "referer": "https://popmart.world.tmall.com/search.htm?spm=a312a.7700824.w4011-15691211895.234.186c5e60uMuZmF&search=y&scene=taobao_shop&orderType=hotsell_desc&viewType=list&pageNo=5&tsearch=y",
    "cookie": "SCF=ApFxOmZR_xsG7Zo91J4ByU7i4IE1sHMFEGNUQRypPgosKbbk66rpjkMbuf-925bBKf_VPkZsMAfxGOHS4F9s__k.; SINAGLOBAL=2000975961750.401.1761899716082; ALF=1766456142; SUB=_2A25EJh4eDeRhGeNM6VsR9ynFyD2IHXVnWh_WrDV8PUJbkNANLUPekW1NTiXseZHXj4yEhaL47eIuxPtbK6hZYR7A; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WhKSsXoBkEgeh2zwP.845hz5JpX5KMhUgL.Fo-Eeo.7S0M4e022dJLoIEnLxKqLBKnLBo2LxKnL122LBo2LxKqL1KnL1KeLxK-L1hqLBoMEe0e0; _s_tentry=-; Apache=738412649430.4799.1763864143978; ULV=1763864143980:2:1:1:738412649430.4799.1763864143978:1761899716083"
}

url = "https://s.weibo.com/weibo?q=%23小甜豆%23"

response = requests.get(url=url, headers=headers)
if response.status_code == 200:
    print("success")
else:
    print("failed...")



success


In [14]:
!pip install selenium
!pip install webdriver-manager  # 这个库可以自动管理浏览器驱动

In [ ]:
############# 小甜豆 ##############
import time
import csv
import re
import random
import os
import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# --- 1. 配置区 ---

MY_COOKIE = "SCF=ApFxOmZR_xsG7Zo91J4ByU7i4IE1sHMFEGNUQRypPgosKbbk66rpjkMbuf-925bBKf_VPkZsMAfxGOHS4F9s__k.; SINAGLOBAL=2000975961750.401.1761899716082; ALF=1766456142; SUB=_2A25EJh4eDeRhGeNM6VsR9ynFyD2IHXVnWh_WrDV8PUJbkNANLUPekW1NTiXseZHXj4yEhaL47eIuxPtbK6hZYR7A; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WhKSsXoBkEgeh2zwP.845hz5JpX5KMhUgL.Fo-Eeo.7S0M4e022dJLoIEnLxKqLBKnLBo2LxKnL122LBo2LxKqL1KnL1KeLxK-L1hqLBoMEe0e0; _s_tentry=-; Apache=738412649430.4799.1763864143978; ULV=1763864143980:2:1:1:738412649430.4799.1763864143978:1761899716083"
URL = "https://s.weibo.com/weibo?q=%23小甜豆%23"
CSV_FILE = 'weibo_xiaotiandou_finish.csv'
MAX_PAGE = 50
DEFAULT_YEAR = "2025"

# --- 2. 核心处理函数 ---

def parse_cookies(cookie_str):
    cookies_dict = []
    for item in cookie_str.split(';'):
        item = item.strip()
        if not item or '=' not in item:
            continue
        try:
            name, value = item.split('=', 1)
            cookies_dict.append({'name': name, 'value': value, 'domain': '.weibo.com'})
        except:
            pass
    return cookies_dict

def extract_number(text):
    match = re.search(r'\d+', text)
    return int(match.group(0)) if match else 0

def process_time_regex(raw_text):
    if not raw_text: return "", "", ""
    text = re.sub(r'\s+', ' ', raw_text).strip()
    now = datetime.datetime.now()

    match_full = re.search(r'(\d{4})年(\d{1,2})月(\d{1,2})日\s*(\d{1,2}:\d{2})', text)
    if match_full: return match_full.group(1), f"{match_full.group(2)}月{match_full.group(3)}日", match_full.group(4)

    match_no_year = re.search(r'(\d{1,2})月(\d{1,2})日\s*(\d{1,2}:\d{2})', text)
    if match_no_year: return DEFAULT_YEAR, f"{match_no_year.group(1)}月{match_no_year.group(2)}日", match_no_year.group(3)

    match_today = re.search(r'今天\s*(\d{1,2}:\d{2})', text)
    if match_today: return str(now.year), f"{now.month}月{now.day}日", match_today.group(1)

    if "分" in text or "秒" in text or "刚刚" in text: return str(now.year), f"{now.month}月{now.day}日", now.strftime("%H:%M")

    match_date_only = re.search(r'(\d{1,2})月(\d{1,2})日', text)
    if match_date_only: return DEFAULT_YEAR, f"{match_date_only.group(1)}月{match_date_only.group(2)}日", "00:00"

    return "", text, "" 

def simulate_human_scroll(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    current_position = 0
    while current_position < last_height:
        step = random.randint(500, 800)
        current_position += step
        driver.execute_script(f"window.scrollTo(0, {current_position});")
        time.sleep(random.uniform(0.3, 0.6))
        new_height = driver.execute_script("return document.body.scrollHeight")
        if current_position > new_height:
            break
        last_height = new_height
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)

def save_to_csv(data, filename):
    file_exists = os.path.isfile(filename)
    try:
        with open(filename, 'a', newline='', encoding='utf-8-sig') as f:
            fieldnames = ['用户名', '帖子内容', '发布年份', '发布日期', '发布时间', '转发数', '评论数', '点赞数']
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            if not file_exists:
                writer.writeheader()
            writer.writerows(data)
    except Exception as e:
        print(f"写入CSV失败: {e}")

# --- 3. 主程序 ---

if __name__ == '__main__':
    chrome_options = Options()
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--start-maximized")
    
    # 【超级隐身设置 1】禁用自动化标志
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    # 【超级隐身设置 2】排除自动化开关
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    # 【调试设置】脚本结束时不自动关闭浏览器（防止闪退后看不到原因）
    chrome_options.add_experimental_option("detach", True)

    chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36")

    print("正在启动浏览器...")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)

    # 【超级隐身设置 3】在页面加载前执行 CDP 命令，修改 navigator.webdriver 为 undefined
    # 这是解决登录闪退、验证码拦截的最关键一步
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
        "source": """
        Object.defineProperty(navigator, 'webdriver', {
            get: () => undefined
        })
        """
    })

    try:
        driver.get("https://weibo.com/")
        time.sleep(3)

        print("正在设置 Cookie...")
        cookies = parse_cookies(MY_COOKIE)
        for cookie in cookies:
            try:
                driver.add_cookie(cookie)
            except:
                continue
        
        driver.get(URL)
        time.sleep(3)

        print("\n" + "="*60)
        print("【稳健登录指南】")
        print("1. 如果页面正常显示：直接按回车。")
        print("2. 如果显示登录框：")
        print("   A. 【推荐】使用微博APP扫码登录 (扫码最稳定，不会闪退)")
        print("   B. 如果必须用短信验证码，现在的代码已加强防闪退，可以尝试。")
        print("="*60 + "\n")
        input("登录完成并准备好后，请按 Enter 键继续...")

        page_count = 1
        
        while True:
            print(f"\n===== 正在爬取第 {page_count} 页 =====")
            
            try:
                WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "div.card-wrap"))
                )
            except TimeoutException:
                print("页面加载超时。")
                break

            simulate_human_scroll(driver)

            try:
                expand_buttons = driver.find_elements(By.XPATH, '//a[@action-type="fl_unfold"]')
                for btn in expand_buttons:
                    try:
                        driver.execute_script("arguments[0].click();", btn)
                        time.sleep(0.5)
                    except:
                        pass
            except:
                pass

            print("   >> 正在解析数据...")
            page_data = []
            posts = driver.find_elements(By.CSS_SELECTOR, "div.card-wrap")
            
            for post in posts:
                try:
                    try:
                        user_name = post.find_element(By.CSS_SELECTOR, 'p.txt').get_attribute('nick-name')
                        if not user_name: continue
                    except:
                        continue

                    content = ""
                    try:
                        content_element = post.find_element(By.CSS_SELECTOR, 'p[node-type="feed_list_content"]')
                        content = content_element.get_attribute('textContent').replace('收起', '').replace('展开', '').strip()
                    except:
                        pass
                    if not content: content = "[仅图片/表情或提取失败]"

                    year, date_part, time_part = "", "", ""
                    try:
                        # 优先使用 wb_time 定位
                        time_element = post.find_element(By.CSS_SELECTOR, 'a[suda-data*="click:wb_time"]')
                        raw_time_text = time_element.get_attribute('textContent')
                        year, date_part, time_part = process_time_regex(raw_time_text)
                    except Exception:
                        # 兜底方案：遍历所有链接
                        try:
                            links = post.find_element(By.CSS_SELECTOR, 'p.from').find_elements(By.TAG_NAME, 'a')
                            for link in links:
                                t = link.get_attribute('textContent')
                                if "月" in t or "今天" in t or "分" in t:
                                    year, date_part, time_part = process_time_regex(t)
                                    break
                        except:
                            pass

                    forwards = comments = likes = 0
                    try:
                        acts = post.find_elements(By.CSS_SELECTOR, 'div.card-act > ul > li')
                        if len(acts) >= 3:
                            forwards = extract_number(acts[0].text)
                            comments = extract_number(acts[1].text)
                            likes = extract_number(acts[2].text)
                    except:
                        pass

                    page_data.append({
                        '用户名': user_name, 
                        '帖子内容': content, 
                        '发布年份': year,
                        '发布日期': date_part,
                        '发布时间': time_part,
                        '转发数': forwards, 
                        '评论数': comments, 
                        '点赞数': likes
                    })
                except:
                    continue

            if page_data:
                save_to_csv(page_data, CSV_FILE)
                print(f"   >> 已保存 {len(page_data)} 条数据。")
            else:
                print("   >> 本页未提取到数据。")

            if page_count >= MAX_PAGE:
                print(f"\n>>> 已完成 {MAX_PAGE} 页任务，程序结束。 <<<")
                break

            try:
                next_btn = driver.find_element(By.CSS_SELECTOR, 'a.next')
                print("   >> 准备翻页...")
                time.sleep(random.uniform(3, 5))
                next_btn.click()
                page_count += 1
                time.sleep(random.uniform(3, 5))
            except NoSuchElementException:
                print("没有下一页了。")
                break
            except Exception as e:
                print(f"翻页出错: {e}")
                break

    except Exception as e:
        print(f"程序出错: {e}")
    
    finally:
        # driver.quit() # 注释掉这一行，让浏览器在出错后保持打开状态，方便你检查
        print("任务结束。")

正在启动浏览器...
正在设置 Cookie...

【稳健登录指南】
1. 如果页面正常显示：直接按回车。
2. 如果显示登录框：
   A. 【推荐】使用微博APP扫码登录 (扫码最稳定，不会闪退)
   B. 如果必须用短信验证码，现在的代码已加强防闪退，可以尝试。


===== 正在爬取第 1 页 =====
   >> 正在解析数据...
   >> 已保存 23 条数据。
   >> 准备翻页...

===== 正在爬取第 2 页 =====
   >> 正在解析数据...
   >> 已保存 9 条数据。
   >> 准备翻页...

===== 正在爬取第 3 页 =====
   >> 正在解析数据...
   >> 已保存 9 条数据。
   >> 准备翻页...

===== 正在爬取第 4 页 =====
   >> 正在解析数据...
   >> 已保存 9 条数据。
   >> 准备翻页...

===== 正在爬取第 5 页 =====
   >> 正在解析数据...
   >> 已保存 9 条数据。
   >> 准备翻页...

===== 正在爬取第 6 页 =====
   >> 正在解析数据...
   >> 已保存 8 条数据。
   >> 准备翻页...

===== 正在爬取第 7 页 =====
   >> 正在解析数据...
   >> 已保存 10 条数据。
   >> 准备翻页...

===== 正在爬取第 8 页 =====
   >> 正在解析数据...
   >> 已保存 10 条数据。
   >> 准备翻页...

===== 正在爬取第 9 页 =====
   >> 正在解析数据...
   >> 已保存 9 条数据。
   >> 准备翻页...

===== 正在爬取第 10 页 =====
   >> 正在解析数据...
   >> 已保存 9 条数据。
   >> 准备翻页...

===== 正在爬取第 11 页 =====
   >> 正在解析数据...
   >> 已保存 8 条数据。
   >> 准备翻页...

===== 正在爬取第 12 页 =====
   >> 正在解析数据...
   >> 已保存 8 条数据。
   >> 准备翻页...

===== 正在爬取第 13 

In [ ]:
############# Dimoo（去重爬取版） ##########
import time
import csv
import re
import random
import os
import datetime
import urllib.parse
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# --- 1. 配置区 ---

RAW_KEYWORD = "#DIMOO#"
CSV_FILE = 'weibo_Dimoo_Future_2025.csv' # 保持文件名一致，自动追加

# 【精准定位续爬】
# 脚本是倒序运行的，所以结束时间设为你的断点日期
START_DATE = "2025-07-01"
END_DATE = "2025-09-21" 

MAX_PAGE_PER_MONTH = 50
DEFAULT_YEAR = "2025"

# --- 2. 核心工具函数 ---

def get_date_ranges(start, end):
    """生成按月切分的时间段"""
    start_date = datetime.datetime.strptime(start, "%Y-%m-%d")
    end_date = datetime.datetime.strptime(end, "%Y-%m-%d")
    ranges = []
    current = start_date
    while current <= end_date:
        if current.month == 12:
            next_month = current.replace(year=current.year+1, month=1, day=1)
        else:
            next_month = current.replace(month=current.month+1, day=1)
        month_end = next_month - datetime.timedelta(days=1)
        
        # 修正最后一个月的时间边界 (这里会把9月修正为9月21日结束)
        if month_end > end_date: 
            month_end = end_date
            
        if current <= month_end:
            ranges.append((current.strftime("%Y-%m-%d"), month_end.strftime("%Y-%m-%d")))
        current = next_month
    
    # 倒序：从 9月 -> 8月 -> 7月 -> 6月
    return ranges[::-1] 

def extract_number(text):
    match = re.search(r'\d+', text)
    return int(match.group(0)) if match else 0

def process_time_regex(raw_text):
    if not raw_text: return "", "", ""
    text = re.sub(r'\s+', ' ', raw_text).strip()
    now = datetime.datetime.now()
    match_full = re.search(r'(\d{4})年(\d{1,2})月(\d{1,2})日\s*(\d{1,2}:\d{2})', text)
    if match_full: return match_full.group(1), f"{match_full.group(2)}月{match_full.group(3)}日", match_full.group(4)
    match_no_year = re.search(r'(\d{1,2})月(\d{1,2})日\s*(\d{1,2}:\d{2})', text)
    if match_no_year: return DEFAULT_YEAR, f"{match_no_year.group(1)}月{match_no_year.group(2)}日", match_no_year.group(3)
    match_today = re.search(r'今天\s*(\d{1,2}:\d{2})', text)
    if match_today: return str(now.year), f"{now.month}月{now.day}日", match_today.group(1)
    if "分" in text or "秒" in text or "刚刚" in text: return str(now.year), f"{now.month}月{now.day}日", now.strftime("%H:%M")
    match_date_only = re.search(r'(\d{1,2})月(\d{1,2})日', text)
    if match_date_only: return DEFAULT_YEAR, f"{match_date_only.group(1)}月{match_date_only.group(2)}日", "00:00"
    return "", text, "" 

def simulate_human_scroll(driver):
    try:
        last_height = driver.execute_script("return document.body.scrollHeight")
        current_position = 0
        while current_position < last_height:
            step = random.randint(600, 900)
            current_position += step
            driver.execute_script(f"window.scrollTo(0, {current_position});")
            time.sleep(random.uniform(0.2, 0.5))
            new_height = driver.execute_script("return document.body.scrollHeight")
            if current_position > new_height: break
            last_height = new_height
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1.0)
    except: pass

def save_to_csv(data, filename):
    file_exists = os.path.isfile(filename)
    try:
        with open(filename, 'a', newline='', encoding='utf-8-sig') as f:
            fieldnames = ['用户名', '帖子内容', '发布年份', '发布日期', '发布时间', '转发数', '评论数', '点赞数', '抓取时段']
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            if not file_exists: writer.writeheader()
            writer.writerows(data)
    except Exception as e:
        print(f"写入CSV失败: {e}")

# --- 3. 浏览器配置 ---

def get_stealth_driver():
    chrome_options = Options()
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    chrome_options.add_experimental_option("detach", True)
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")

    driver = webdriver.Chrome(options=chrome_options)
    
    # 设置超时防断连
    driver.set_page_load_timeout(120)
    driver.set_script_timeout(120)

    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
        "source": """
            Object.defineProperty(navigator, 'webdriver', { get: () => undefined });
            Object.defineProperty(navigator, 'plugins', { get: () => [1, 2, 3, 4, 5] });
            Object.defineProperty(navigator, 'languages', { get: () => ['zh-CN', 'zh', 'en'] });
        """
    })
    return driver

# --- 4. 主程序 ---

if __name__ == '__main__':
    print(f"正在启动续爬模式 (目标: {START_DATE} ~ {END_DATE})...")
    
    try:
        driver = get_stealth_driver()
    except Exception as e:
        print("\n启动失败，请升级Selenium")
        exit()

    seen_posts = set()

    try:
        # --- A. 登录 ---
        driver.get("https://weibo.com/")
        print("\n" + "="*60)
        print("【请手动扫码登录】")
        print("登录成功看到首页后，回到这里按 Enter 键。")
        print("="*60 + "\n")
        input("准备好后，请按 Enter 键...")

        # --- B. 热身 ---
        print("正在热身...")
        encoded_keyword = urllib.parse.quote(RAW_KEYWORD)
        try:
            driver.get(f"https://s.weibo.com/weibo?q={encoded_keyword}&Refer=index")
            simulate_human_scroll(driver)
            time.sleep(random.uniform(2, 4))
        except TimeoutException:
            print("热身超时，跳过...")

        # --- C. 续爬开始 ---
        date_ranges = get_date_ranges(START_DATE, END_DATE)
        print(f"任务计划：剩余 {len(date_ranges)} 个时间段。")

        for start_t, end_t in date_ranges:
            search_url = f"https://s.weibo.com/weibo?q={encoded_keyword}&typeall=1&suball=1&timescope=custom:{start_t}:{end_t}&Refer=g"
            
            print(f"\n>>> 正在抓取: {start_t} 至 {end_t} <<<")
            try:
                driver.get(search_url)
                time.sleep(random.uniform(3, 5)) 
            except TimeoutException:
                print("   加载超时，刷新...")
                driver.refresh()
                time.sleep(5)

            if "抱歉，未找到" in driver.page_source:
                print(f"   无内容，跳过。")
                continue

            page_count = 1
            while True:
                print(f"   第 {page_count} 页...", end="")
                
                try:
                    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.card-wrap")))
                except:
                    print(" -> 等待超时")

                simulate_human_scroll(driver)

                try:
                    expand_buttons = driver.find_elements(By.XPATH, '//a[@action-type="fl_unfold"]')
                    for btn in expand_buttons:
                        driver.execute_script("arguments[0].click();", btn)
                        time.sleep(0.1)
                except: pass

                page_data = []
                posts = driver.find_elements(By.CSS_SELECTOR, "div.card-wrap")
                duplicate_count = 0
                
                for post in posts:
                    try:
                        if "card-no-result" in post.get_attribute("class"): continue
                        try:
                            user_name = post.find_element(By.CSS_SELECTOR, 'p.txt').get_attribute('nick-name')
                        except: continue
                        
                        content = ""
                        try:
                            content = post.find_element(By.CSS_SELECTOR, 'p[node-type="feed_list_content"]').get_attribute('textContent').replace('收起', '').replace('展开', '').strip()
                        except: pass
                        if not content: content = "[内容提取失败]"

                        year, date_part, time_part = "", "", ""
                        try:
                            time_element = post.find_element(By.CSS_SELECTOR, 'a[suda-data*="click:wb_time"]')
                            year, date_part, time_part = process_time_regex(time_element.get_attribute('textContent'))
                        except:
                            try:
                                links = post.find_element(By.CSS_SELECTOR, 'p.from').find_elements(By.TAG_NAME, 'a')
                                for link in links:
                                    if "月" in link.text or "今天" in link.text:
                                        year, date_part, time_part = process_time_regex(link.text)
                                        break
                            except: pass

                        # 去重
                        unique_id = f"{user_name}_{content[:30]}_{date_part}_{time_part}"
                        if unique_id in seen_posts:
                            duplicate_count += 1
                            continue 
                        seen_posts.add(unique_id)

                        forwards = comments = likes = 0
                        try:
                            acts = post.find_elements(By.CSS_SELECTOR, 'div.card-act > ul > li')
                            if len(acts) >= 3:
                                forwards = extract_number(acts[0].text)
                                comments = extract_number(acts[1].text)
                                likes = extract_number(acts[2].text)
                        except: pass

                        page_data.append({
                            '用户名': user_name, '帖子内容': content, 
                            '发布年份': year, '发布日期': date_part, '发布时间': time_part,
                            '转发数': forwards, '评论数': comments, '点赞数': likes,
                            '抓取时段': f"{start_t}_{end_t}"
                        })
                    except: continue

                if page_data:
                    save_to_csv(page_data, CSV_FILE)
                    print(f" -> 保存 {len(page_data)} 条 (去重 {duplicate_count} 条)")
                else:
                    print(f" -> 本页无新数据 (可能已存过)")

                if page_count >= MAX_PAGE_PER_MONTH:
                    print("   月度上限，下个月。")
                    break

                try:
                    next_btn = driver.find_element(By.CSS_SELECTOR, 'a.next')
                    time.sleep(random.uniform(3, 5))
                    next_btn.click()
                    page_count += 1
                    time.sleep(random.uniform(3, 5))
                except:
                    print("   无下一页。")
                    break
            
            time.sleep(random.uniform(3, 5))

    except Exception as e:
        print(f"程序异常: {e}")

正在启动续爬模式 (目标: 2025-06-01 ~ 2025-09-21)...

【请手动扫码登录】
登录成功看到首页后，回到这里按 Enter 键。

正在热身...
任务计划：剩余 4 个时间段。

>>> 正在抓取: 2025-09-01 至 2025-09-21 <<<
   第 1 页... -> 保存 10 条 (去重 0 条)
   第 2 页... -> 保存 10 条 (去重 0 条)
   第 3 页... -> 保存 8 条 (去重 0 条)
   第 4 页... -> 保存 10 条 (去重 0 条)
   第 5 页... -> 保存 10 条 (去重 0 条)
   第 6 页... -> 保存 10 条 (去重 0 条)
   第 7 页... -> 保存 10 条 (去重 0 条)
   第 8 页... -> 保存 10 条 (去重 0 条)
   第 9 页... -> 保存 10 条 (去重 0 条)
   第 10 页... -> 保存 9 条 (去重 0 条)
   第 11 页... -> 保存 1 条 (去重 0 条)
   无下一页。

>>> 正在抓取: 2025-08-01 至 2025-08-31 <<<
   第 1 页... -> 保存 10 条 (去重 0 条)
   第 2 页... -> 保存 10 条 (去重 0 条)
   第 3 页... -> 保存 10 条 (去重 0 条)
   第 4 页... -> 保存 10 条 (去重 0 条)
   第 5 页... -> 保存 10 条 (去重 0 条)
   第 6 页... -> 保存 9 条 (去重 0 条)
   第 7 页... -> 保存 10 条 (去重 0 条)
   第 8 页... -> 保存 10 条 (去重 0 条)
   第 9 页... -> 保存 9 条 (去重 0 条)
   第 10 页... -> 保存 10 条 (去重 0 条)
   第 11 页... -> 保存 9 条 (去重 0 条)
   第 12 页... -> 保存 9 条 (去重 0 条)
   第 13 页... -> 保存 8 条 (去重 0 条)
   第 14 页... -> 保存 10 条 (去重 0 条)
   第 15 页..

In [ ]:
############# Molly ##########
import time
import csv
import re
import random
import os
import datetime
import urllib.parse
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# --- 1. 配置区 ---

RAW_KEYWORD = "#Molly#" 
CSV_FILE = 'weibo_Molly_Specific_2025.csv'

START_DATE = "2025-07-01"
END_DATE = "2025-11-26" 

MAX_PAGE_PER_MONTH = 50
DEFAULT_YEAR = "2025"

# --- 2. 核心工具函数 ---

def get_date_ranges(start, end):
    start_date = datetime.datetime.strptime(start, "%Y-%m-%d")
    end_date = datetime.datetime.strptime(end, "%Y-%m-%d")
    ranges = []
    current = start_date
    while current <= end_date:
        if current.month == 12:
            next_month = current.replace(year=current.year+1, month=1, day=1)
        else:
            next_month = current.replace(month=current.month+1, day=1)
        month_end = next_month - datetime.timedelta(days=1)
        if month_end > end_date: month_end = end_date
        if current <= month_end:
            ranges.append((current.strftime("%Y-%m-%d"), month_end.strftime("%Y-%m-%d")))
        current = next_month
    return ranges[::-1] 

def extract_number(text):
    match = re.search(r'\d+', text)
    return int(match.group(0)) if match else 0

def process_time_regex(raw_text):
    if not raw_text: return "", "", ""
    text = re.sub(r'\s+', ' ', raw_text).strip()
    now = datetime.datetime.now()
    match_full = re.search(r'(\d{4})年(\d{1,2})月(\d{1,2})日\s*(\d{1,2}:\d{2})', text)
    if match_full: return match_full.group(1), f"{match_full.group(2)}月{match_full.group(3)}日", match_full.group(4)
    match_no_year = re.search(r'(\d{1,2})月(\d{1,2})日\s*(\d{1,2}:\d{2})', text)
    if match_no_year: return DEFAULT_YEAR, f"{match_no_year.group(1)}月{match_no_year.group(2)}日", match_no_year.group(3)
    match_today = re.search(r'今天\s*(\d{1,2}:\d{2})', text)
    if match_today: return str(now.year), f"{now.month}月{now.day}日", match_today.group(1)
    if "分" in text or "秒" in text or "刚刚" in text: return str(now.year), f"{now.month}月{now.day}日", now.strftime("%H:%M")
    match_date_only = re.search(r'(\d{1,2})月(\d{1,2})日', text)
    if match_date_only: return DEFAULT_YEAR, f"{match_date_only.group(1)}月{match_date_only.group(2)}日", "00:00"
    return "", text, "" 

def simulate_human_scroll(driver):
    """模拟下滑加载"""
    # 必须先滑到底，因为微博是懒加载，不滑到底下面的帖子HTML元素都不存在
    try:
        driver.execute_script("window.stop();") 
        last_height = driver.execute_script("return document.body.scrollHeight")
        current_position = 0
        while current_position < last_height:
            step = random.randint(500, 800)
            current_position += step
            driver.execute_script(f"window.scrollTo(0, {current_position});")
            time.sleep(random.uniform(0.2, 0.4))
            new_height = driver.execute_script("return document.body.scrollHeight")
            if current_position > new_height: break
            last_height = new_height
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1.0)
    except: pass

def save_to_csv(data, filename):
    file_exists = os.path.isfile(filename)
    try:
        with open(filename, 'a', newline='', encoding='utf-8-sig') as f:
            fieldnames = ['用户名', '帖子内容', '发布年份', '发布日期', '发布时间', '转发数', '评论数', '点赞数', '抓取时段']
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            if not file_exists: writer.writeheader()
            writer.writerows(data)
    except Exception as e:
        print(f"写入CSV失败: {e}")

# --- 3. 浏览器配置 ---

def get_stealth_driver():
    chrome_options = Options()
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    chrome_options.add_experimental_option("detach", True)
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")

    driver = webdriver.Chrome(options=chrome_options)
    driver.set_page_load_timeout(15)
    driver.set_script_timeout(15)

    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
        "source": """
            Object.defineProperty(navigator, 'webdriver', { get: () => undefined });
            Object.defineProperty(navigator, 'plugins', { get: () => [1, 2, 3, 4, 5] });
            Object.defineProperty(navigator, 'languages', { get: () => ['zh-CN', 'zh', 'en'] });
        """
    })
    return driver

# --- 4. 主程序 ---

if __name__ == '__main__':
    print(f"正在启动 (目标: {START_DATE} ~ {END_DATE})...")
    
    try:
        driver = get_stealth_driver()
    except Exception as e:
        print("\n启动失败，请升级Selenium")
        exit()

    seen_posts = set()

    try:
        driver.get("https://weibo.com/")
        print("\n" + "="*60)
        print("【请手动扫码登录】")
        print("登录成功看到首页后，回到这里按 Enter 键。")
        print("="*60 + "\n")
        input("准备好后，请按 Enter 键...")

        print("已跳过热身，直接开始...")

        date_ranges = get_date_ranges(START_DATE, END_DATE)
        print(f"任务计划：{len(date_ranges)} 个时段。")

        for start_t, end_t in date_ranges:
            encoded_keyword = urllib.parse.quote(RAW_KEYWORD)
            search_url = f"https://s.weibo.com/weibo?q={encoded_keyword}&typeall=1&suball=1&timescope=custom:{start_t}:{end_t}&Refer=g&nodup=1"
            
            print(f"\n>>> 正在抓取: {start_t} 至 {end_t} <<<")
            
            try:
                driver.get(search_url)
                time.sleep(random.uniform(3, 5))
            except TimeoutException:
                print("   页面加载超时(>15s)，强制停止加载并开始解析...")
                driver.execute_script("window.stop();") 

            if "抱歉，未找到" in driver.page_source:
                print(f"   微博提示：未找到相关结果。")
                continue

            page_count = 1
            while True:
                print(f"   第 {page_count} 页...", end="")
                
                try:
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.card-wrap")))
                except:
                    print(" -> 等待超时")

                # 必须先下滑，确保帖子加载出来，否则 find_elements 找不到
                simulate_human_scroll(driver)

                page_data = []
                # 获取本页所有帖子元素
                posts = driver.find_elements(By.CSS_SELECTOR, "div.card-wrap")
                duplicate_count = 0
                
                # --- 针对每条帖子单独处理 ---
                for post in posts:
                    try:
                        if "card-no-result" in post.get_attribute("class"): continue
                        
                        # 1. 【关键修改】在处理当前帖子前，先检查并点击它内部的展开按钮
                        try:
                            # 使用相对 XPath (.//) 查找当前 post 内部的展开按钮
                            expand_btn = post.find_element(By.XPATH, './/a[@action-type="fl_unfold"]')
                            driver.execute_script("arguments[0].click();", expand_btn)
                            time.sleep(0.1) # 稍等一下，让内容展开
                        except:
                            # 如果没有展开按钮，或者报错，说明不需要展开，直接继续
                            pass

                        # 2. 开始抓取信息
                        try:
                            user_name = post.find_element(By.CSS_SELECTOR, 'p.txt').get_attribute('nick-name')
                        except: continue
                        
                        content = ""
                        try:
                            content = post.find_element(By.CSS_SELECTOR, 'p[node-type="feed_list_content"]').get_attribute('textContent').replace('收起', '').replace('展开', '').strip()
                        except: pass
                        if not content: content = "[内容提取失败]"

                        year, date_part, time_part = "", "", ""
                        try:
                            time_element = post.find_element(By.CSS_SELECTOR, 'a[suda-data*="click:wb_time"]')
                            year, date_part, time_part = process_time_regex(time_element.get_attribute('textContent'))
                        except:
                            try:
                                links = post.find_element(By.CSS_SELECTOR, 'p.from').find_elements(By.TAG_NAME, 'a')
                                for link in links:
                                    if "月" in link.text or "今天" in link.text:
                                        year, date_part, time_part = process_time_regex(link.text)
                                        break
                            except: pass

                        unique_id = f"{user_name}_{content[:30]}_{date_part}_{time_part}"
                        if unique_id in seen_posts:
                            duplicate_count += 1
                            continue 
                        seen_posts.add(unique_id)

                        forwards = comments = likes = 0
                        try:
                            acts = post.find_elements(By.CSS_SELECTOR, 'div.card-act > ul > li')
                            if len(acts) >= 3:
                                forwards = extract_number(acts[0].text)
                                comments = extract_number(acts[1].text)
                                likes = extract_number(acts[2].text)
                        except: pass

                        page_data.append({
                            '用户名': user_name, '帖子内容': content, 
                            '发布年份': year, '发布日期': date_part, '发布时间': time_part,
                            '转发数': forwards, '评论数': comments, '点赞数': likes,
                            '抓取时段': f"{start_t}_{end_t}"
                        })

                    except: continue

                if page_data:
                    save_to_csv(page_data, CSV_FILE)
                    print(f" -> 保存 {len(page_data)} 条 (去重 {duplicate_count} 条)")
                else:
                    print(f" -> 本页无新数据")

                if page_count >= MAX_PAGE_PER_MONTH:
                    print("   月度翻页上限，切换下月。")
                    break

                try:
                    next_btn = driver.find_element(By.CSS_SELECTOR, 'a.next')
                    time.sleep(random.uniform(3, 5))
                    next_btn.click()
                    page_count += 1
                    time.sleep(random.uniform(3, 5))
                except:
                    print("   无下一页。")
                    break
            
            time.sleep(random.uniform(3, 5))

    except Exception as e:
        print(f"程序异常: {e}")

正在启动 (目标: 2025-07-01 ~ 2025-11-26)...

【请手动扫码登录】
登录成功看到首页后，回到这里按 Enter 键。

已跳过热身，直接开始...
任务计划：5 个时段。

>>> 正在抓取: 2025-11-01 至 2025-11-26 <<<
   第 1 页... -> 保存 10 条 (去重 0 条)
   第 2 页... -> 保存 10 条 (去重 0 条)
   第 3 页... -> 保存 6 条 (去重 0 条)
   无下一页。

>>> 正在抓取: 2025-10-01 至 2025-10-31 <<<
   第 1 页... -> 保存 10 条 (去重 0 条)
   第 2 页... -> 保存 10 条 (去重 0 条)
   第 3 页... -> 保存 9 条 (去重 1 条)
   第 4 页... -> 保存 10 条 (去重 0 条)
   第 5 页... -> 保存 9 条 (去重 0 条)
   无下一页。

>>> 正在抓取: 2025-09-01 至 2025-09-30 <<<
   第 1 页... -> 保存 9 条 (去重 0 条)
   第 2 页... -> 保存 10 条 (去重 0 条)
   第 3 页... -> 保存 10 条 (去重 0 条)
   第 4 页... -> 保存 9 条 (去重 0 条)
   第 5 页... -> 保存 9 条 (去重 0 条)
   第 6 页... -> 保存 1 条 (去重 0 条)
   无下一页。

>>> 正在抓取: 2025-08-01 至 2025-08-31 <<<
   第 1 页... -> 保存 10 条 (去重 0 条)
   第 2 页... -> 保存 9 条 (去重 0 条)
   第 3 页... -> 保存 9 条 (去重 0 条)
   第 4 页... -> 保存 8 条 (去重 0 条)
   第 5 页... -> 保存 10 条 (去重 0 条)
   第 6 页... -> 保存 9 条 (去重 0 条)
   第 7 页... -> 保存 10 条 (去重 0 条)
   第 8 页... -> 保存 9 条 (去重 0 条)
   第 9 页... -> 保存 6 条 (去

In [5]:
############## Skullpanda ############
import time
import csv
import re
import random
import os
import datetime
import urllib.parse
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# --- 1. 配置区 (已更新为 Skullpanda) ---

# 关键词
RAW_KEYWORD = "#skullpanda#" 
# 文件名
CSV_FILE = 'weibo_Skullpanda_2025_Full.csv'

# 时间范围：2025 全年
START_DATE = "2025-01-01"
END_DATE = "2025-12-31" 

MAX_PAGE_PER_MONTH = 50
DEFAULT_YEAR = "2025"

# --- 2. 核心工具函数 ---

def get_date_ranges(start, end):
    start_date = datetime.datetime.strptime(start, "%Y-%m-%d")
    end_date = datetime.datetime.strptime(end, "%Y-%m-%d")
    now = datetime.datetime.now()
    if end_date > now: end_date = now
    ranges = []
    current = start_date
    while current <= end_date:
        if current.month == 12:
            next_month = current.replace(year=current.year+1, month=1, day=1)
        else:
            next_month = current.replace(month=current.month+1, day=1)
        month_end = next_month - datetime.timedelta(days=1)
        if month_end > end_date: month_end = end_date
        if current <= month_end:
            ranges.append((current.strftime("%Y-%m-%d"), month_end.strftime("%Y-%m-%d")))
        current = next_month
    # 倒序：从最近的月份开始爬
    return ranges[::-1] 

def extract_number(text):
    match = re.search(r'\d+', text)
    return int(match.group(0)) if match else 0

def process_time_regex(raw_text):
    if not raw_text: return "", "", ""
    text = re.sub(r'\s+', ' ', raw_text).strip()
    now = datetime.datetime.now()
    match_full = re.search(r'(\d{4})年(\d{1,2})月(\d{1,2})日\s*(\d{1,2}:\d{2})', text)
    if match_full: return match_full.group(1), f"{match_full.group(2)}月{match_full.group(3)}日", match_full.group(4)
    match_no_year = re.search(r'(\d{1,2})月(\d{1,2})日\s*(\d{1,2}:\d{2})', text)
    if match_no_year: return DEFAULT_YEAR, f"{match_no_year.group(1)}月{match_no_year.group(2)}日", match_no_year.group(3)
    match_today = re.search(r'今天\s*(\d{1,2}:\d{2})', text)
    if match_today: return str(now.year), f"{now.month}月{now.day}日", match_today.group(1)
    if "分" in text or "秒" in text or "刚刚" in text: return str(now.year), f"{now.month}月{now.day}日", now.strftime("%H:%M")
    match_date_only = re.search(r'(\d{1,2})月(\d{1,2})日', text)
    if match_date_only: return DEFAULT_YEAR, f"{match_date_only.group(1)}月{match_date_only.group(2)}日", "00:00"
    return "", text, "" 

def simulate_human_scroll(driver):
    try:
        last_height = driver.execute_script("return document.body.scrollHeight")
        current_position = 0
        while current_position < last_height:
            step = random.randint(500, 800)
            current_position += step
            driver.execute_script(f"window.scrollTo(0, {current_position});")
            time.sleep(random.uniform(0.3, 0.6))
            new_height = driver.execute_script("return document.body.scrollHeight")
            if current_position > new_height: break
            last_height = new_height
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1.5)
    except: pass

def save_to_csv(data, filename):
    file_exists = os.path.isfile(filename)
    try:
        with open(filename, 'a', newline='', encoding='utf-8-sig') as f:
            fieldnames = ['用户名', '帖子内容', '发布年份', '发布日期', '发布时间', '转发数', '评论数', '点赞数', '抓取时段']
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            if not file_exists: writer.writeheader()
            writer.writerows(data)
    except Exception as e:
        print(f"写入CSV失败: {e}")

# --- 3. CDP 隐身浏览器配置 ---

def get_stealth_driver():
    chrome_options = Options()
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    chrome_options.add_experimental_option("detach", True)
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")

    # 使用原生驱动管理，避免路径报错
    driver = webdriver.Chrome(options=chrome_options)

    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
        "source": """
            Object.defineProperty(navigator, 'webdriver', { get: () => undefined });
            Object.defineProperty(navigator, 'plugins', { get: () => [1, 2, 3, 4, 5] });
            Object.defineProperty(navigator, 'languages', { get: () => ['zh-CN', 'zh', 'en'] });
            const getParameter = WebGLRenderingContext.prototype.getParameter;
            WebGLRenderingContext.prototype.getParameter = function(parameter) {
                if (parameter === 37445) { return 'Intel Open Source Technology Center'; }
                if (parameter === 37446) { return 'Mesa DRI Intel(R) Ivybridge Mobile '; }
                return getParameter(parameter);
            };
        """
    })
    return driver

# --- 4. 主程序 ---

if __name__ == '__main__':
    print("正在启动隐身模式浏览器 (Skullpanda 2025版)...")
    
    try:
        driver = get_stealth_driver()
    except Exception as e:
        print("\n启动失败！请尝试升级 Selenium: pip install --upgrade selenium")
        print(f"详细错误: {e}")
        exit()

    try:
        driver.get("https://weibo.com/")
        print("\n" + "="*60)
        print("【请手动扫码登录】")
        print("1. 请使用手机微博APP扫码。")
        print("2. 登录成功并看到微博首页后，回到这里按 Enter 键。")
        print("="*60 + "\n")
        input("准备好后，请按 Enter 键...")

        print("正在进行热身搜索...")
        encoded_keyword = urllib.parse.quote(RAW_KEYWORD)
        driver.get(f"https://s.weibo.com/weibo?q={encoded_keyword}&Refer=index")
        simulate_human_scroll(driver)
        time.sleep(random.uniform(3, 5))
        print("热身完成，开始抓取 2025 年数据。")

        date_ranges = get_date_ranges(START_DATE, END_DATE)
        print(f"任务计划：分为 {len(date_ranges)} 个月度时段。")

        for start_t, end_t in date_ranges:
            search_url = f"https://s.weibo.com/weibo?q={encoded_keyword}&typeall=1&suball=1&timescope=custom:{start_t}:{end_t}&Refer=g"
            
            print(f"\n>>> 正在抓取时段: {start_t} 至 {end_t} <<<")
            driver.get(search_url)
            time.sleep(random.uniform(4, 6)) 

            if "抱歉，未找到" in driver.page_source:
                print(f"   该时段无内容，跳过。")
                continue

            page_count = 1
            while True:
                print(f"   第 {page_count} 页...")
                try:
                    WebDriverWait(driver, 8).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.card-wrap")))
                except:
                    print("   等待超时 (可能无内容)。")

                simulate_human_scroll(driver)

                try:
                    expand_buttons = driver.find_elements(By.XPATH, '//a[@action-type="fl_unfold"]')
                    for btn in expand_buttons:
                        driver.execute_script("arguments[0].click();", btn)
                        time.sleep(0.2)
                except: pass

                page_data = []
                posts = driver.find_elements(By.CSS_SELECTOR, "div.card-wrap")
                
                for post in posts:
                    try:
                        if "card-no-result" in post.get_attribute("class"): continue
                        try:
                            user_name = post.find_element(By.CSS_SELECTOR, 'p.txt').get_attribute('nick-name')
                        except: continue
                        
                        content = ""
                        try:
                            content = post.find_element(By.CSS_SELECTOR, 'p[node-type="feed_list_content"]').get_attribute('textContent').replace('收起', '').replace('展开', '').strip()
                        except: pass
                        if not content: content = "[内容提取失败或仅图片]"

                        year, date_part, time_part = "", "", ""
                        try:
                            time_element = post.find_element(By.CSS_SELECTOR, 'a[suda-data*="click:wb_time"]')
                            year, date_part, time_part = process_time_regex(time_element.get_attribute('textContent'))
                        except:
                            try:
                                links = post.find_element(By.CSS_SELECTOR, 'p.from').find_elements(By.TAG_NAME, 'a')
                                for link in links:
                                    if "月" in link.text or "今天" in link.text:
                                        year, date_part, time_part = process_time_regex(link.text)
                                        break
                            except: pass

                        forwards = comments = likes = 0
                        try:
                            acts = post.find_elements(By.CSS_SELECTOR, 'div.card-act > ul > li')
                            if len(acts) >= 3:
                                forwards = extract_number(acts[0].text)
                                comments = extract_number(acts[1].text)
                                likes = extract_number(acts[2].text)
                        except: pass

                        page_data.append({
                            '用户名': user_name, '帖子内容': content, 
                            '发布年份': year, '发布日期': date_part, '发布时间': time_part,
                            '转发数': forwards, '评论数': comments, '点赞数': likes,
                            '抓取时段': f"{start_t}_{end_t}"
                        })
                    except: continue

                if page_data:
                    save_to_csv(page_data, CSV_FILE)
                    print(f"   保存 {len(page_data)} 条。")
                else:
                    print("   本页未提取到有效数据。")

                if page_count >= MAX_PAGE_PER_MONTH:
                    print("   达到单月页数上限，切换下一月。")
                    break

                try:
                    next_btn = driver.find_element(By.CSS_SELECTOR, 'a.next')
                    time.sleep(random.uniform(3, 5))
                    next_btn.click()
                    page_count += 1
                    time.sleep(random.uniform(3, 5))
                except:
                    print("   无下一页。")
                    break
            
            time.sleep(random.uniform(4, 6))

    except Exception as e:
        print(f"程序异常: {e}")

正在启动隐身模式浏览器 (Skullpanda 2025版)...

【请手动扫码登录】
1. 请使用手机微博APP扫码。
2. 登录成功并看到微博首页后，回到这里按 Enter 键。

正在进行热身搜索...
热身完成，开始抓取 2025 年数据。
任务计划：分为 11 个月度时段。

>>> 正在抓取时段: 2025-11-01 至 2025-11-26 <<<
   第 1 页...
   保存 10 条。
   第 2 页...
   保存 10 条。
   第 3 页...
   保存 10 条。
   第 4 页...
   保存 10 条。
   第 5 页...
   保存 8 条。
   第 6 页...
   保存 10 条。
   第 7 页...
   保存 10 条。
   第 8 页...
   保存 10 条。
   第 9 页...
   保存 9 条。
   第 10 页...
   保存 10 条。
   第 11 页...
   保存 3 条。
   无下一页。

>>> 正在抓取时段: 2025-10-01 至 2025-10-31 <<<
   第 1 页...
   保存 10 条。
   第 2 页...
   保存 10 条。
   第 3 页...
   保存 10 条。
   第 4 页...
   保存 10 条。
   第 5 页...
   保存 10 条。
   第 6 页...
   保存 9 条。
   无下一页。

>>> 正在抓取时段: 2025-09-01 至 2025-09-30 <<<
   第 1 页...
   保存 10 条。
   第 2 页...
   保存 9 条。
   第 3 页...
   保存 10 条。
   第 4 页...
   保存 10 条。
   第 5 页...
   保存 10 条。
   第 6 页...
   保存 10 条。
   第 7 页...
   保存 10 条。
   第 8 页...
   保存 10 条。
   第 9 页...
   保存 10 条。
   第 10 页...
   保存 10 条。
   第 11 页...
   保存 10 条。
   第 12 页...
   保存 10 条。
   第 13 页...
   保存 